## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [204]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from collections import Counter

%matplotlib inline

In [205]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [206]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [211]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self,
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None,
                 debug=False):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.debug = debug
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            raise ValueError('invalid criterion name')

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            raise ValueError('invalid max_features name')

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # 1. Мера неопределенности родительского узла
        i_p = self.__gini_p((l_c + r_c) / (l_s + r_s))
        # 2. Мера неопределенности левого дочернего узла
        i_l = self.__gini_p(l_c / l_s)
        # 3. Мера неопределенности правого дочернего узла
        i_r = self.__gini_p(r_c / r_s)
        return self.__impurity(i_p, i_l, i_r, l_s, r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.sqrt(n_feature)))
        return feature_ids[:max_features]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.log2(n_feature)))
        return feature_ids[:max_features]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем значения свойства по возрастанию и приводим к аналогичной последовательности целевые переменные.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Количество меток класса.
        class_number = np.unique(y).shape[0]

        if self.debug:
            print('sorted_x', sorted_x)
            print('sorted_y', sorted_y)
            print('class_number', class_number)
        
        # Здесь, по всей видимости, мы обрезаем образцы с учетом параметра min_samples_split, для того, 
        # чтобы в середине осталось необходимое количество образцов размера min_samples_split. 
        # Хотя это немного странный подход, можно проверять min_samples_split заранее.
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # Сравниваем каждый элемент с предыдущим, таким образом мы получаем информацию о том насколько неоднородная выборка,
        # это будет иметь влияния на вычисление меры неоднородности.
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if self.debug:
            print('splitted_sorted_y', splitted_sorted_y)
            print('r_border_ids', r_border_ids)
        
        # Если выборка однородная, то нет необходимости делать расщепление.
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что происходит деталях объяснить не так легко, но судя по отладочной информации, 
        # основной смысл данного участка кода - сформировать различные варианты разбиения классов, 
        # на основе которых можно вычислить различные меры неоднородности и выбрать из них наиболее оптимальную.
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] \
            + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)

        if self.debug:
            print('eq_el_count', eq_el_count)
            print('one_hot_code', one_hot_code)
            print('class_increments', class_increments)
        
        # Здесь создаются различные варианты разбиения классов по узлам дерева:
        # l_class_count - варианты разбиения для левого узла
        # r_class_count - варианты разбиения для правого узла
        # l_sizes - общее число образцов в левом узле
        # r_sizes - общее число образцов в правом узле
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        if self.debug:
            print('gs', gs)
            print('idx', idx)
    
        # Что делает этот блок кода?
        left_el_id = l_sizes[idx][0]

        if self.debug:
            print('left_el_id', left_el_id)
            print()

        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):        
        # Если узел содержит только образцы одного класса, то останавливаем расщепление.
        if np.unique(y).shape[0] == 1:
            self.tree[node_id] = (self.LEAF_TYPE, y[0], 0,)
            return

        # Если доля образцов одного класса больше или равна необходимой доли для расщепления, то останавливаем расщепление.
        most_common_y = Counter(y).most_common(1).pop()
        if most_common_y[1] >= self.sufficient_share * y.shape[0]:
            self.tree[node_id] = (self.LEAF_TYPE, most_common_y[0], 0,)
            return

        # Если достигнута максимальная глубина дерева, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if self.max_depth == depth:
            self.tree[node_id] = (self.LEAF_TYPE, most_common_y[0], 0,)
            return

        # Если количество образцов меньше требуемого для разделения узла, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if y.shape[0] < min_samples_split:
            self.tree[node_id] = (self.LEAF_TYPE, most_common_y[0], 0,)
            return
        
        n_samples, n_features = x.shape
        split_data = []
        # Ищем признак по которому будем делать разбиение (который ведет к самому большому приросту информации).
        # Для этого необходимо вычислить для каждого признака меру неоднородности (impurity).
        # Чем ниже неоднородность, тем выше прирост информации.
        # Также здесь мы вычисляем порог (threshold), 
        # по которому будем определять идти в правый узел дерева или левый (т.к. дерево у нас бинарное).
        for feature_id in self.get_feature_ids(n_features):
            impurity, threshold = self.__find_threshold(x[:,feature_id], y)
            if threshold is not None:
                split_data.append((impurity, threshold, feature_id,))

        # Если недостаточно данных для разбиения, например, 
        # достигнут предел минимального количества образцов, то останавливаем расщепление.
        # Делаем предсказание по классу с наибольшим количеством образцов.
        if not split_data:
            self.tree[node_id] = (self.LEAF_TYPE, most_common_y[0], 0,)
            return

        best_split = min(split_data, key=lambda x: x[0])

        if self.debug:
            print('best_split', {
                'impurity': best_split[0],
                'threshold': best_split[1],
                'feature_id': best_split[2],
            })
            print()

        # Если в узле нет данных, значит для этого узла мы не можем создать лист.
        if x.shape[0] == 0 or y.shape[0] == 0:
            return
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    @staticmethod
    def __gini_p(p):
        return 1 - (p ** 2).sum(axis=1)

    @staticmethod
    def __entropy_p(p):
        return - np.nansum(p * np.log2(p), axis=1)

    @staticmethod
    def __misclass_p(p):   
        return 1 - p.max(axis=1)

    @staticmethod
    def __impurity(i_p, i_l, i_r, l_s, r_s):
        t_s = l_s + r_s
        return i_p - (np.squeeze(np.asarray(l_s / t_s)) * i_l) - (np.squeeze(np.asarray(r_s / t_s)) * i_r)


In [212]:
df_debug = df.sample(n=50)
x_debug = df_debug.as_matrix(columns=df_debug.columns[1:])
y_debug = df_debug.as_matrix(columns=df_debug.columns[:1])
y_debug = y_debug.reshape(y_debug.shape[0])
my_clf = MyDecisionTreeClassifier(min_samples_split=2, debug=True)
my_clf.fit(x_debug, y_debug)

sorted_x [ 0.          0.          0.          0.          0.00167948  0.00338263
  0.00390843  0.01206496  0.01422296  0.01803303  0.01882488  0.02238107
  0.02633568  0.02843239  0.03107158  0.0367378   0.06015076  0.0644897
  0.06466236  0.08012148  0.08711252  0.08964776  0.09126137  0.09362382
  0.09971776  0.14566507  0.14920039  0.16922147  0.18200621  0.21755649
  0.2581584   0.27882807  0.32489001  0.38218054  0.39390303  0.52182559
  0.58496742  0.63264091  0.66908458  0.67309369  0.7496809   0.77552257
  0.79282869  0.94978751  0.96220756  0.96815459  0.98056635  0.99750063
  1.00419916  1.06930238]
sorted_y [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 1 0 0 0 1 1]
class_number 2
splitted_sorted_y [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 1 0 0 0]
r_border_ids [35 36 41 42 44 45]
eq_el_count [33  1  5  1  2  1]
one_hot_code [[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.

In [213]:
np.unique(np.array([0,0,0,1,2])).shape[0]

3

In [44]:
t = np.array([0,0,0,0,0,0,0,0,0,0,0])
Counter(t).most_common(1).pop()

(0, 11)

In [49]:
data = [(1, 5, 3), (2, 3, 8), (7, 6, 1)]
min(data, key = lambda t: t[1])

(2, 3, 8)

In [45]:
d = np.array([3, 3, 1, 2, 1, 2, 3, 25, 4])
d_y = np.bincount(d)
print(d)
d_y

[ 3  3  1  2  1  2  3 25  4]


array([0, 2, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1])

In [48]:
y.shape[0]

100

In [191]:
e = np.array([0.66, 0.7, 0.8, 0.82])
v = np.array([0, 0.1077551, 0.095, 0.13563355])
e * v

array([ 0.        ,  0.07542857,  0.076     ,  0.11121951])

In [51]:
a = np.array([6, 8, 2, 7, 1, 10, 6, 8, 2]).reshape(3, 3)
a

array([[ 6,  8,  2],
       [ 7,  1, 10],
       [ 6,  8,  2]])

In [52]:
a[:,1]

array([8, 1, 8])

In [53]:
a_idx = a[:,1].argsort()
a_idx

array([1, 0, 2])

In [54]:
a[a_idx]

array([[ 7,  1, 10],
       [ 6,  8,  2],
       [ 6,  8,  2]])

In [55]:
a[1:]

array([[ 7,  1, 10],
       [ 6,  8,  2]])

In [56]:
a[:-1]

array([[ 6,  8,  2],
       [ 7,  1, 10]])

In [98]:
min_samples_split = 2
b = np.array([0,0,1,1,0,1,0,0,1])
class_number = np.unique(b).shape[0]
b_split = b[min_samples_split:-min_samples_split]
print(b)
print(b_split)
r_border_ids = np.where(b_split[:-1] != b_split[1:])[0] + (min_samples_split + 1)
print(r_border_ids)
print(r_border_ids[:-1])
print(np.append([min_samples_split], r_border_ids[:-1]))
eq_el_count = r_border_ids - np.append([min_samples_split], r_border_ids[:-1])
print(eq_el_count)
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
print(one_hot_code)
one_hot_code[np.arange(r_border_ids.shape[0]), b[r_border_ids - 1]] = 1
print(one_hot_code)
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)

[0 0 1 1 0 1 0 0 1]
[1 1 0 1 0]
[4 5 6]
[4 5]
[2 4 5]
[2 1 1]
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]


In [16]:
np.log2(4)
c = np.array([6, 8, 2, 1, 7, 1, 4, 10, 6, 8, 2, 1])

array([6])

In [29]:
x.argsort()[1]

array([2, 6, 7, 8, 3, 0, 9, 5, 1, 4])

In [46]:
np.unique(y).size

2

In [64]:
def __sort_samples(x, y):
    sorted_idx = x.argsort()
    return x[sorted_idx], y[sorted_idx]

In [65]:
sorted_x, sorted_y = __sort_samples(x, y)

In [93]:
splitted_sorted_y = sorted_y[2:-2]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (2 + 1)
splitted_sorted_y[1:]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(x, y)

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.00365805625916
1.13050794601


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

1.0
1.0
0.999958426873
1.0
0.999958425144


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.894321110834
0.895859316538
0.891078406918
0.890828968155
0.891406477362


# Применить для задачи Titanic 